In [28]:
import pandas as pd
import pandas as pd
import spacy 
#TODO: !pip3 install spacy
#TODO: !python3 -m spacy download en_core_web_sm
from sklearn.metrics.pairwise import cosine_similarity
import re

In [29]:
#Reading the csv
csv_file_path = 'UpdatedResumeDataSet.csv'
resumes_df = pd.read_csv(csv_file_path)
resumes_df.head()

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


In [30]:
#analyzing data
resumes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 962 entries, 0 to 961
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  962 non-null    object
 1   Resume    962 non-null    object
dtypes: object(2)
memory usage: 15.2+ KB


In [31]:
#looking at possible values for the category column
unique_classes = resumes_df['Category'].unique()
print(unique_classes)

['Data Science' 'HR' 'Advocate' 'Arts' 'Web Designing'
 'Mechanical Engineer' 'Sales' 'Health and fitness' 'Civil Engineer'
 'Java Developer' 'Business Analyst' 'SAP Developer' 'Automation Testing'
 'Electrical Engineering' 'Operations Manager' 'Python Developer'
 'DevOps Engineer' 'Network Security Engineer' 'PMO' 'Database' 'Hadoop'
 'ETL Developer' 'DotNet Developer' 'Blockchain' 'Testing']


In [32]:
# Load pre-trained English tokenizer, tagger, parser, NER, and word vectors
nlp = spacy.load("en_core_web_sm")

def preprocess_text(text):
    # Remove non-alphanumeric characters and extra whitespaces
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    
    # Tokenize text and remove stop words
    doc = nlp(text)
    tokens = [token.text.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

def screen_resumes(job_description, category, resumes_df, top_n=10):
    # Filter resumes by category
    category_resumes = resumes_df[resumes_df['Category'] == category].copy()  # Make a copy to avoid modifying the original DataFrame
    
    # Preprocess job description and category resumes
    job_description_processed = preprocess_text(job_description)
    category_resumes['Resume_Processed'] = category_resumes['Resume'].apply(preprocess_text)
    
    # Calculate word embeddings for job description
    job_description_embedding = nlp(job_description_processed).vector
    
    # Calculate word embeddings for resumes and calculate cosine similarity
    similarities = []
    for resume_text in category_resumes['Resume_Processed']:
        resume_embedding = nlp(resume_text).vector
        similarity = cosine_similarity([job_description_embedding], [resume_embedding])[0][0]
        similarities.append(similarity)
    
    # Add cosine similarity scores to dataframe using .loc
    category_resumes.loc[:, 'Similarity'] = similarities
    
    # Sort resumes by similarity and select top n
    top_resumes = category_resumes.sort_values(by='Similarity', ascending=False).head(top_n)
    
    return top_resumes


In [33]:
# Example job description and category
job_description = "We are looking for a data scientist proficient in Python, machine learning, and data visualization."
category = "Data Science"

In [34]:
# Screen resumes for the job description and category
top_resumes = screen_resumes(job_description, category, resumes_df)
top_resumes

,Category,Resume,Resume_Processed,Similarity
0,Data Science,Skills * Programming Languages: Python (pandas...,skills programming languages python pandas n...,0.826909
10,Data Science,Skills * Programming Languages: Python (pandas...,skills programming languages python pandas n...,0.826909
30,Data Science,Skills * Programming Languages: Python (pandas...,skills programming languages python pandas n...,0.826909
20,Data Science,Skills * Programming Languages: Python (pandas...,skills programming languages python pandas n...,0.826909
18,Data Science,Personal Skills â¢ Ability to quickly grasp t...,personal skills ability quickly grasp techni...,0.795275
38,Data Science,Personal Skills â¢ Ability to quickly grasp t...,personal skills ability quickly grasp techni...,0.795275
8,Data Science,Personal Skills â¢ Ability to quickly grasp t...,personal skills ability quickly grasp techni...,0.795275
28,Data Science,Personal Skills â¢ Ability to quickly grasp t...,personal skills ability quickly grasp techni...,0.795275
39,Data Science,Expertise â Data and Quantitative Analysis â...,expertise data quantitative analysis decis...,0.780576
9,Data Science,Expertise â Data and Quantitative Analysis â...,expertise data quantitative analysis decis...,0.780576
